## Building a Question-Answering Chatbot with Gradio

In this exercise, we will use LangChain and OpenAI's ChatGPT [API](https://platform.openai.com/docs/guides/gpt) to build a chatbot that can answer questions using information from a set of source documents.

#### Part 1: Chatbot Interface
First let's build the interface using Gradio and get the skeleton of the chatbot in place. 

For the time being, we will use `send_chat_message()` to simulate the chatbot's responses.

In [ ]:
import gradio as gr
import random

def send_chat_message(prompt):
    return random.choice(["How are you?", "Have a great day!", "I'm very hungry"])

def respond(message, chat_history):
    bot_message = send_chat_message(message)
    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks(title = "✨ UB DocBot ✨") as demo:
    with gr.Row():
        gr.Image(value="assets/unbounce-identity-blue.png").style(height=75)
        gr.HTML("""
        <img src="assets/unbounce-identity-blue.png"></div>
        """)
        gr.Markdown(
        """
        # Hello World!
        Welcome to ✨ *UB DocBot* ✨, an intelligent chatbot that will help find answers to your deepest, most burning questions about the Unbounce universe.
        """)
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="What question can I help you with today?")
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
demo.launch()

#### Part 2: The AI Chatbot
Now we can build the AI chatbot component. We shall use OpenAI's chat API.

##### Using Environment Files
In the last demo, we put our secret key in plaintext in the notebook itself. This is not very secure. If we wanted to share this notebook with someone else, we would always have to remember to remove the secret key.

An alternative is to use environment files and simply use the notebook to read the values from that file. To do this,
1. Create a new blank file in the same directory and call it `.env`
2. Write `OPENAI_API_KEY=<your secret key>`
3. Now we can read these environment variables using `load_dotenv()`

In [ ]:
import openai
import os
from dotenv import load_dotenv

# Load API keys
load_dotenv()
openai.organization = os.getenv('OPENAI_ORG_KEY')
openai.api_key = os.getenv('OPENAI_API_KEY')

##### LangChain
LangChain is a framework for building apps using language models. There are many off-the-shelf chains - sequences of calls to components, including other chains - that make it easy to get started. We will use the `ConversationalRetrievalChain()`.


In [ ]:
import sys

from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores import Chroma

DATA_DIRECTORY = "data"
VECTOR_STORE_PATH = os.path.join(DATA_DIRECTORY, "vectors")

def load_unstructured_pdf(path):
    """
    https://python.langchain.com/docs/modules/data_connection/document_loaders/how_to/pdf#using-unstructured
    """
    loader = UnstructuredPDFLoader(path)
    documents = loader.load()
    return documents


def load_embeddings_from_store(path):
    """Load persisted embeddings."""
    vectorstore = Chroma(
        persist_directory=path, embedding_function=OpenAIEmbeddings()
    )
    return VectorStoreIndexWrapper(vectorstore=vectorstore)


def create_embeddings(document_path, directory):
    """Embedding"""
    documents = load_unstructured_pdf(document_path)
    return VectorstoreIndexCreator(
        vectorstore_kwargs={"persist_directory": directory}
    ).from_documents(documents)

def get_embeddings():
    """
    https://python.langchain.com/docs/modules/chains/popular/chat_vector_db
    https://github.com/techleadhd/chatgpt-retrieval/blob/main/chatgpt.py
    """
    if os.path.exists(VECTOR_STORE_PATH):
        return load_embeddings_from_store()
    else:
        document_path = os.path.join(DATA_DIRECTORY, "Content Library 2023-07-06.pdf")
        return create_embeddings(document_path, VECTOR_STORE_PATH)

In [ ]:
QA_PROMPT_TEMPLATE = """Use the following pieces of context to answer the question at the end. Make sure your answer is as detailed as possible. Be witty and humourous, but always be positive and encouraging. If you don't know the answer, say a joke in response.

{context}

Question: {question}
Helpful Answer:"""

def send_chat_message(prompt, chat_history, chain):
    """
    Converts Gradio's chat history format to LangChain's expected 
    format and queries the chat engine for a response.
    """
    langchain_history = [(msg[0], msg[1]) for msg in chat_history]

    result = chain({"question": prompt, "chat_history": langchain_history})
    print(f"The original question is: {prompt} and the generated question is: {result['generated_question']}")
    return result["answer"]

def respond(message, chat_history):
    bot_message = send_chat_message(message, chat_history, chain)
    chat_history.append((message, bot_message))
    return "", chat_history

In [ ]:
index = get_embeddings()
qa_prompt = PromptTemplate.from_template(template=QA_PROMPT_TEMPLATE)
chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model="gpt-4", temperature=0.9),
    retriever=index.vectorstore.as_retriever(search_kwargs={"k": 1}),
    condense_question_llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo'),
    return_generated_question=True,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

In [ ]:
with gr.Blocks(title = "UB DocBot") as demo:
    with gr.Row():
        gr.Markdown(
        """
        <h1 align='left'> <img src='file/assets/unbounce-identity-blue.png' height='34' width='129'> </h1>
        <h1 align="center"> DocBot 🤖💬 </h1>
        <p align='center' style='font-size: 18px;'> Meet ✨ DocBot ✨, an intelligent chatbot that helps answer your deepest questions about the Unbounce universe. </p>
        """)
    chatbot = gr.Chatbot(elem_id="ub_docbot")
    msg = gr.Textbox(placeholder="Have a question? How can I help? Start the chat.", show_label=False)
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    gr.Examples(
        examples=["How do I cancel my plan?",
                    "How do I upgrade plans?",
                    "How do I add my domain to Unbounce?",
                    "How do I connect my Wordpress domain to Unbounce?",
                    "I want to transfer my domain to another account, how can I do that?",
                    "How do I add a user to my account?",
                    "How does Smart Traffic compare to A/B testing?",
                    "What happens when I exceed account limits on my Optimize plan?",
                    "Do I get access to Smart Builder with the Optimize plan?",
                    "I'm currently on the annual 'Optimize plan', how do I cancel?"
                    ],
        inputs=msg
    )
demo.launch(share=True, server_name="0.0.0.0", server_port=7860)